In [ ]:
import os
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import cv2
from glob import glob
import numpy as np
import datetime
import skimage.measure
import json
import skimage
from skimage.measure import find_contours, approximate_polygon

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
#     if keep_ratio:
#         im.thumbnail((size, size), resample)
#     else:
#         im = im.resize((size, size), resample)
    
    return im

In [ ]:
for split in ['test']:
    load_dir = f'../input/body-morphometry-kidney-and-tumor/{split}/DICOM/'
    save_dir = f'/kaggle/tmp/{split}/'

    os.makedirs(save_dir, exist_ok=True)

    for path in tqdm(os.listdir(load_dir)):
        # set keep_ratio=True to have original aspect ratio
        for file in os.listdir(os.path.join(load_dir, path)):
            xray = read_xray(os.path.join(load_dir, path, file))
            im = resize(xray)  
            im.save(os.path.join(save_dir, path + file).replace('dcm', 'png'))

In [ ]:
import torch
from torch.utils.data import Dataset,DataLoader

class KidneyDataset(Dataset):
    
    def __init__(self, label_dir, transform=None):
        
        self.label_dirs = sorted(glob(label_dir))
        self.transform = transform
        
    def __getitem__(self,index):
        
        img = cv2.imread(self.label_dirs[index])
        img = img.astype(np.float32)
        
        if self.transform is not None:
            transformed = self.transform(image=img)
            img = transformed['image']
            
        return self.label_dirs[index], img
        
    def __len__(self):
        return len(self.label_dirs)

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

test_transform = A.Compose([
    A.Normalize(),
    ToTensorV2()
])

test_dataset = KidneyDataset('/kaggle/tmp/test/*', transform = test_transform)
test_loader = DataLoader(dataset=test_dataset)

In [ ]:
import sys
sys.path.append('../input/segmentation-modelspytorch')

import segmentation_models_pytroch as smp

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = smp.Unet(encoder_name='tu-efficientnetv2_rw_m', encoder_weights = None, classes=3 , activation=None)
model = model.to(device)

model_path = '../input/efficientnet-baseline/effi2unet_baseline.pt'

checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['state_dict'])

In [ ]:
def rle_encode(mask_image):
    pixels = mask_image.flatten()
    # We avoid issues with '1' at the start or end (at the corners of
    # the original image) by setting those pixels to '0' explicitly.
    # We do not expect these to be non-zero for an accurate mask,
    # so this should not harm the score.
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return runs


def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

In [ ]:
f = open('./submission.csv', "w")
f.write("Id,EncodedPixels\n")

model.eval()

print('Start prediction...')

for i, (info, img) in tqdm(enumerate(test_loader), total = len(test_loader)):

    if i % 64 == 0:
        data_id_1 = f'test{format((i//64) + 1, "03")}_{1}'
        data_id_2 = f'test{format((i//64) + 1, "03")}_{2}'

        mask_stack1 = np.array([], dtype='uint8')
        mask_stack2 = np.array([], dtype='uint8')
            
    with torch.no_grad():
        # imgs - 1,3,512,512
        
        outs = model(img.to(device)) # 1,3,512,512
        outs = torch.argmax(outs, dim=1).detach().cpu().numpy() # (1,512,512)
        
        mask_stack1 = np.hstack([mask_stack1, np.array(outs[0] == 1).flatten()])
        mask_stack2 = np.hstack([mask_stack2, np.array(outs[0] == 2).flatten()])
    
    if (i + 1) % 64==0:
        enc = rle_to_string(rle_encode(mask_stack1))
        line = f'{data_id_1},{enc}'
        f.write(line + '\n')
        
        enc = rle_to_string(rle_encode(mask_stack2))
        line = f'{data_id_2},{enc}'
        f.write(line + '\n')
        
f.close()

print('DONE!')